In [1]:
import uuid
from typing import Dict, Any, Optional
from dotenv import load_dotenv

In [2]:
!pwd

/Users/evergreenllc/code/adp


In [3]:
load_dotenv()

True

In [4]:
!python3 -m pip install --upgrade google-adk

  Using cached google_adk-1.19.0-py3-none-any.whl.metadata (13 kB)
  Using cached aiosqlite-0.21.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached authlib-1.6.5-py2.py3-none-any.whl.metadata (9.8 kB)
  Using cached fastapi-0.118.3-py3-none-any.whl.metadata (28 kB)
  Using cached google_api_python_client-2.187.0-py3-none-any.whl.metadata (7.0 kB)
  Using cached google_cloud_aiplatform-1.128.0-py2.py3-none-any.whl.metadata (46 kB)
  Using cached google_cloud_bigquery_storage-2.34.0-py3-none-any.whl.metadata (10 kB)
  Using cached google_cloud_bigtable-2.34.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached google_cloud_discoveryengine-0.13.12-py3-none-any.whl.metadata (9.6 kB)
  Using cached google_cloud_secret_manager-2.25.0-py3-none-any.whl.metadata (9.8 kB)
  Using cached google_cloud_spanner-3.59.0-py3-none-any.whl.metadata (11 kB)
  Using cached google_cloud_speech-2.34.0-py3-none-any.whl.metadata (9.8 kB)
  Using cached google_cloud_storage-3.6.0-py3-none-any.whl.metadata (13 kB

In [5]:
import sys
print(sys.executable)
print(sys.version)

/Users/evergreenllc/code/adp/adk_env/bin/python3
3.11.9 (v3.11.9:de54cf5be3, Apr  2 2024, 07:12:50) [Clang 13.0.0 (clang-1300.0.29.30)]


In [6]:
!python3.14 -m pip show google-adk

Name: google-adk
Version: 1.19.0
Summary: Agent Development Kit
Home-page: https://google.github.io/adk-docs/
Author: 
Author-email: Google LLC <googleapis-packages@google.com>
License: 
Location: /Library/Frameworks/Python.framework/Versions/3.14/lib/python3.14/site-packages
Requires: aiosqlite, anyio, authlib, click, fastapi, google-api-python-client, google-cloud-aiplatform, google-cloud-bigquery, google-cloud-bigquery-storage, google-cloud-bigtable, google-cloud-discoveryengine, google-cloud-secret-manager, google-cloud-spanner, google-cloud-speech, google-cloud-storage, google-genai, graphviz, jsonschema, mcp, opentelemetry-api, opentelemetry-exporter-gcp-logging, opentelemetry-exporter-gcp-monitoring, opentelemetry-exporter-gcp-trace, opentelemetry-exporter-otlp-proto-http, opentelemetry-resourcedetector-gcp, opentelemetry-sdk, pyarrow, pydantic, python-dateutil, python-dotenv, PyYAML, requests, sqlalchemy, sqlalchemy-spanner, starlette, tenacity, typing-extensions, tzlocal, uvic

In [6]:
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.adk.tools import FunctionTool
from google.genai import types
from google.adk.events import Event

In [8]:
import google.adk
print(google.adk.__version__)

1.19.0


In [3]:

import uuid
from typing import Dict, Any, Optional
from dotenv import load_dotenv
from google.adk.tools import FunctionTool

# Cell 2: Load environment
load_dotenv()



True

In [4]:
# Cell 4: Create Handler Functions and Wrap as Tools
def handle_technical_support(issue_description: str) -> str:
    """
    Process a technical support request and provide troubleshooting assistance.
    
    Args:
        issue_description: Description of the technical issue the customer is experiencing
        
    Returns:
        A response with troubleshooting steps or resolution
    """
    print(" ---  Technical Support Tool Called  ---")
    print(f"     Issue: {issue_description}")
    return f"Technical support ticket created for: {issue_description}"


def handle_billing_support(billing_query: str) -> str:
    """
    Process a billing support request and handle payment-related issues.
    
    Args:
        billing_query: Description of the billing issue or question
        
    Returns:
        A response addressing the billing concern
    """
    print(" ---  Billing Support Tool Called  ---")
    print(f"     Query: {billing_query}")
    return f"Billing support processing: {billing_query}"


# Wrap functions as FunctionTools
technical_tool = FunctionTool(handle_technical_support)
billing_tool = FunctionTool(handle_billing_support)

In [7]:
# Cell 5: Create Technical Support Agent
technical_agent = Agent(
    name="technical_agent",
    model="gemini-2.0-flash",
    instruction=(
        "You are a technical support specialist. "
        "Help customers troubleshoot software issues, bugs, and technical problems. "
        "Use the handle_technical_support tool to process each technical issue. "
        "Provide clear step-by-step solutions and ask clarifying questions when needed."
    ),
    description="Handles all technical support requests including bugs and troubleshooting.",
    tools=[technical_tool]
)

# Cell 6: Create Billing Support Agent
billing_agent = Agent(
    name="billing_agent",
    model="gemini-2.0-flash",
    instruction=(
        "You are a billing support specialist. "
        "Assist customers with payment issues, subscription questions, refunds, and invoices. "
        "Use the handle_billing_support tool to process each billing request. "
        "Be empathetic and resolve financial concerns professionally."
    ),
    description="Provides billing support, handles payments, and manages subscription issues.",
    tools=[billing_tool]
)

In [11]:
# Cell 7: Create Coordinator
coordinator = Agent(
    name="coordinator",
    model="gemini-2.0-flash",
    instruction=(
        "You are a support coordinator responsible for routing customer requests to the appropriate specialist. "
        "Analyze the customer's request and determine which agent should handle it:\n\n"
        "- For technical issues (bugs, crashes, errors, features not working), "
        "transfer to 'technical_agent'\n"
        "- For billing matters (payments, charges, refunds, subscriptions, invoices), "
        "transfer to 'billing_agent'\n\n"
        "Do not attempt to solve the requests yourself. Your job is solely to route to the correct agent."
    ),
    description="Main coordinator that routes customer requests to specialized support agents.",
    sub_agents=[technical_agent, billing_agent]
)


In [15]:
async def run_coordinator(runner:InMemoryRunner, request:str):
    """
        Runs the coordinator agent with a given request and delegates.
    """
    print(f"\n --- Running Coordinator with request: '{request}'--- ")
    final_result = ""
    try:
        user_id = "user_123"
        session_id = str(uuid.uuid4())
        await runner.session_service.create_session(app_name=runner.app_name, 
                                                    session_id = session_id,
                                                    user_id = user_id,
                                                   )
        for event in runner.run(
                                 user_id = user_id,
                                 session_id = session_id,
                                 new_message = types.Content(
                                                             role='user',
                                                             parts=[types.Part(text=request)]
                                         
                                                             )
                                ):

            if event.is_final_response() and event.content:
                if hasattr(event.content, "text") and event.content.text:
                    final_result = event.content.text
                elif event.content.parts:
                    text_parts = [part.text for part in event.content.parts if part.text]
                    final_result = "".join(text_parts)
                break
        print(f"Coordinator Final response :{final_result}")
        return final_result
    except Exception as e:
        print(f"An error occurred while processing your request:{e}")
        return f"An error occurred while processing your request '{request}' \n Error: {e}"


In [16]:
# Cell 9: Main test function
async def main():
    print("-- Google ADK Customer Support Routing Example -- ")
    runner = InMemoryRunner(coordinator)
    
    result_a = await run_coordinator(runner, "My app crashes every time I try to login")
    print(f"final output A : {result_a}")

    result_b = await run_coordinator(runner, "Why was I charged $99 instead of $49?")
    print(f"final output B : {result_b}")

    result_c = await run_coordinator(runner, "The export feature is not working correctly")
    print(f"final output C : {result_c}")

    result_d = await run_coordinator(runner, "I need to cancel my subscription and get a refund")
    print(f"final output D : {result_d}")


In [17]:
# Cell 10: Jupyter compatibility
import nest_asyncio

# Cell 11: Apply patch
nest_asyncio.apply()

# Cell 12: Run
await main()

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/evergreenllc/code/adp/adk_env/lib/python3.11/site-packages/google/adk/agents", which implies app name "agents".


-- Google ADK Customer Support Routing Example -- 

 --- Running Coordinator with request: 'My app crashes every time I try to login'--- 


Coordinator Final response :Please provide more details about the issue. What type of device are you using (e.g., iPhone, Android, Windows)? What version of the app do you have installed? Also, can you describe what happens right before the crash? Are there any error messages?

final output A : Please provide more details about the issue. What type of device are you using (e.g., iPhone, Android, Windows)? What version of the app do you have installed? Also, can you describe what happens right before the crash? Are there any error messages?


 --- Running Coordinator with request: 'Why was I charged $99 instead of $49?'--- 
 ---  Billing Support Tool Called  ---
     Query: Why was I charged $99 instead of $49?
Coordinator Final response :I can look into why you were charged $99 instead of $49. To best assist you, could you please provide more information, such as your account details or the date of the charge? This will help me investigate the issue and provide you with an accurate exp